In [58]:
# NOTE: do a binary log reg using the statistics gathered above. 
# Maybe do an ensemble method, using the predicted probability from a naive bayes as a feature?
# look into de-duplication

# find which terms return no results, remove
# then check which papers mention any of atp words OR any of stimulation words (grouped together)
# do a quick scatter plot visualizing atp mentions vs mechanical word mentions
# ADD SINGLE TERMS TO QUERY LIST


In [59]:
# look into stanford core nlp

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
import seaborn as sn
import matplotlib.pyplot as plt
import collections
import operator
import warnings; warnings.simplefilter('ignore')
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score
from sklearn.model_selection import StratifiedKFold
from IPython.core.interactiveshell import InteractiveShell
import plotly.plotly as py
import plotly.tools as tls

import matplotlib.pyplot as plt
# InteractiveShell.ast_node_interactivity = "all"

C:\Users\Tanner\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [9]:
text_df = pd.read_pickle("data/data_frame_no_paper.pkl")



In [4]:
with open('data/atprelease_query_terms1.pkl', 'rb') as f:
    query_terms1 = pickle.load(f)
    
with open('data/atprelease_query_terms2.pkl', 'rb') as f:
    query_terms2 = pickle.load(f)

In [5]:

def sort_dict_by_values(dictionary, min_freq):
    sorted_list = [(k, dictionary[k]) for k in sorted(dictionary, key=dictionary.get, reverse=True)]
    
    
    sorted_dict = {}
    for e in sorted_list[:1000]:
        if e[1] >= min_freq:
            sorted_dict[e[0]] = e[1]
    
    return sorted_dict

In [6]:
def normalize(normalizer, dictionary):
    new_dict = {}
    for key in dictionary.keys():
        if np.around(dictionary[key]/normalizer, 3) > 0:
            new_dict[key] = np.around(dictionary[key]/normalizer, 3)
    return new_dict

In [7]:
def get_num_relevant(df):
    counts = df['relevant'].value_counts()
    relevant = counts[1]
    irrelevant  = counts[0]
    return relevant, irrelevant

In [10]:
# get various tallies of all words

title_tally_R    = {}
keyword_tally_R  = {}
abstract_tally_R = {}
corpus_tally_R   = {}

title_tally_NR    = {}
keyword_tally_NR  = {}
abstract_tally_NR = {}
corpus_tally_NR   = {}

for row in text_df.iterrows():
    row = row[1]
    
    relevant        = row.relevant
    title_corpus    = row.title_corpus
    keyword_corpus  = row.keyword_corpus
    abstract_corpus = row.abstract_corpus
    total_corpus    = row.all_text_corpus
    
        
    if relevant:
        
        for word in title_corpus:            
            if word in title_tally_R:
                title_tally_R[word] += 1                
            else:
                title_tally_R[word]  = 1
            
            if word in corpus_tally_R:
                corpus_tally_R[word] += 1                
            else:
                corpus_tally_R[word]  = 1                
        
        for word in keyword_corpus:            
            if word in keyword_tally_R:
                keyword_tally_R[word] += 1                
            else:
                keyword_tally_R[word]  = 1
            
            if word in corpus_tally_R:
                corpus_tally_R[word] += 1                
            else:
                corpus_tally_R[word]  = 1
        
        for word in abstract_corpus:            
            if word in abstract_tally_R:
                abstract_tally_R[word] += 1                
            else:
                abstract_tally_R[word]  = 1
            
            if word in corpus_tally_R:
                corpus_tally_R[word] += 1                
            else:
                corpus_tally_R[word]  = 1
                
    else:
        
        for word in title_corpus:            
            if word in title_tally_NR:
                title_tally_NR[word] += 1                
            else:
                title_tally_NR[word]  = 1
            
            if word in corpus_tally_NR:
                corpus_tally_NR[word] += 1                
            else:
                corpus_tally_NR[word]  = 1                
        
        for word in keyword_corpus:            
            if word in keyword_tally_NR:
                keyword_tally_NR[word] += 1                
            else:
                keyword_tally_NR[word]  = 1
            
            if word in corpus_tally_NR:
                corpus_tally_NR[word] += 1                
            else:
                corpus_tally_NR[word]  = 1
        
        for word in abstract_corpus:            
            if word in abstract_tally_NR:
                abstract_tally_NR[word] += 1                
            else:
                abstract_tally_NR[word]  = 1
            
            if word in corpus_tally_NR:
                corpus_tally_NR[word] += 1                
            else:
                corpus_tally_NR[word]  = 1   

In [11]:
title_tally_R      = sort_dict_by_values(title_tally_R, 1)
keyword_tally_R    = sort_dict_by_values(keyword_tally_R, 1)
abstract_tally_R   = sort_dict_by_values(abstract_tally_R, 1)
corpus_tally_R     = sort_dict_by_values(corpus_tally_R, 1)

title_tally_NR     = sort_dict_by_values(title_tally_NR, 1)
keyword_tally_NR   = sort_dict_by_values(keyword_tally_NR, 1)
abstract_tally_NR  = sort_dict_by_values(abstract_tally_NR, 1)
corpus_tally_NR    = sort_dict_by_values(corpus_tally_NR, 1)

In [12]:
# get various tallies of all query terms

qt1_title_tally_R    = {}
qt1_keyword_tally_R  = {}
qt1_abstract_tally_R = {}
qt1_corpus_tally_R   = {}

qt1_title_tally_NR    = {}
qt1_keyword_tally_NR  = {}
qt1_abstract_tally_NR = {}
qt1_corpus_tally_NR   = {}

for row in text_df.iterrows():
    row = row[1]
    relevant        = row.relevant
    title_corpus    = row.title_corpus
    keyword_corpus  = row.keyword_corpus
    abstract_corpus = row.abstract_corpus
    total_corpus    = row.all_text_corpus        

        
    for word in title_corpus:

        if word in query_terms1:
            if relevant:
                if word in qt_title_tally_R:
                    qt_title_tally_R[word] += 1                
                else:
                    qt_title_tally_R[word]  = 1

                if word in qt_corpus_tally_R:
                    qt_corpus_tally_R[word] += 1                
                else:
                    qt_corpus_tally_R[word]  = 1      
            else:
                if word in qt_title_tally_NR:
                    qt_title_tally_NR[word] += 1                
                else:
                    qt_title_tally_NR[word]  = 1

                if word in qt_corpus_tally_NR:
                    qt_corpus_tally_NR[word] += 1                
                else:
                    qt_corpus_tally_NR[word]  = 1   

    for word in keyword_corpus:  

        if word in query_terms1:
            if relevant:
                if word in qt_keyword_tally_R:
                    qt_keyword_tally_R[word] += 1                
                else:
                    qt_keyword_tally_R[word]  = 1

                if word in qt_corpus_tally_R:
                    qt_corpus_tally_R[word] += 1                
                else:
                    qt_corpus_tally_R[word]  = 1
            else:
                if word in qt_keyword_tally_NR:
                    qt_keyword_tally_NR[word] += 1                
                else:
                    qt_keyword_tally_NR[word]  = 1

                if word in qt_corpus_tally_NR:
                    qt_corpus_tally_NR[word] += 1                
                else:
                    qt_corpus_tally_NR[word]  = 1


    for word in abstract_corpus:  

        if word in query_terms1:
            if relevant:
                if word in qt_abstract_tally_R:
                    qt_abstract_tally_R[word] += 1                
                else:
                    qt_abstract_tally_R[word]  = 1

                if word in qt_corpus_tally_R:
                    qt_corpus_tally_R[word] += 1                
                else:
                    qt_corpus_tally_R[word]  = 1
            else:

                if word in qt_abstract_tally_NR:
                    qt_abstract_tally_NR[word] += 1                
                else:
                    qt_abstract_tally_NR[word]  = 1

                if word in qt_corpus_tally_NR:
                    qt_corpus_tally_NR[word] += 1                
                else:
                    qt_corpus_tally_NR[word]  = 1


NameError: name 'qt_title_tally_R' is not defined

In [13]:
# get various tallies of all query terms

qt2_title_tally_R    = {}
qt2_keyword_tally_R  = {}
qt2_abstract_tally_R = {}
qt2_corpus_tally_R   = {}

qt2_title_tally_NR    = {}
qt2_keyword_tally_NR  = {}
qt2_abstract_tally_NR = {}
qt2_corpus_tally_NR   = {}

for row in text_df.iterrows():
    row = row[1]
    relevant        = row.relevant
    title_corpus    = row.title_corpus
    keyword_corpus  = row.keyword_corpus
    abstract_corpus = row.abstract_corpus
    total_corpus    = row.all_text_corpus        

        
    for word in title_corpus:

        if word in query_terms2:
            if relevant:
                if word in qt_title_tally_R:
                    qt_title_tally_R[word] += 1                
                else:
                    qt_title_tally_R[word]  = 1

                if word in qt_corpus_tally_R:
                    qt_corpus_tally_R[word] += 1                
                else:
                    qt_corpus_tally_R[word]  = 1      
            else:
                if word in qt_title_tally_NR:
                    qt_title_tally_NR[word] += 1                
                else:
                    qt_title_tally_NR[word]  = 1

                if word in qt_corpus_tally_NR:
                    qt_corpus_tally_NR[word] += 1                
                else:
                    qt_corpus_tally_NR[word]  = 1   

    for word in keyword_corpus:  

        if word in query_terms2:
            if relevant:
                if word in qt_keyword_tally_R:
                    qt_keyword_tally_R[word] += 1                
                else:
                    qt_keyword_tally_R[word]  = 1

                if word in qt_corpus_tally_R:
                    qt_corpus_tally_R[word] += 1                
                else:
                    qt_corpus_tally_R[word]  = 1
            else:
                if word in qt_keyword_tally_NR:
                    qt_keyword_tally_NR[word] += 1                
                else:
                    qt_keyword_tally_NR[word]  = 1

                if word in qt_corpus_tally_NR:
                    qt_corpus_tally_NR[word] += 1                
                else:
                    qt_corpus_tally_NR[word]  = 1


    for word in abstract_corpus:  

        if word in query_terms2:
            if relevant:
                if word in qt_abstract_tally_R:
                    qt_abstract_tally_R[word] += 1                
                else:
                    qt_abstract_tally_R[word]  = 1

                if word in qt_corpus_tally_R:
                    qt_corpus_tally_R[word] += 1                
                else:
                    qt_corpus_tally_R[word]  = 1
            else:

                if word in qt_abstract_tally_NR:
                    qt_abstract_tally_NR[word] += 1                
                else:
                    qt_abstract_tally_NR[word]  = 1

                if word in qt_corpus_tally_NR:
                    qt_corpus_tally_NR[word] += 1                
                else:
                    qt_corpus_tally_NR[word]  = 1


NameError: name 'qt_title_tally_R' is not defined

In [14]:
qt1_title_tally_R      = sort_dict_by_values(qt1_title_tally_R, 2)
qt1_keyword_tally_R    = sort_dict_by_values(qt1_keyword_tally_R, 2)
qt1_abstract_tally_R   = sort_dict_by_values(qt1_abstract_tally_R, 2)
qt1_corpus_tally_R     = sort_dict_by_values(qt1_corpus_tally_R, 2)

qt1_title_tally_NR     = sort_dict_by_values(qt1_title_tally_NR, 2)
qt1_keyword_tally_NR   = sort_dict_by_values(qt1_keyword_tally_NR, 2)
qt1_abstract_tally_NR  = sort_dict_by_values(qt1_abstract_tally_NR, 2)
qt1_corpus_tally_NR    = sort_dict_by_values(qt1_corpus_tally_NR, 2)


In [94]:
qt2_title_tally_R      = sort_dict_by_values(qt2_title_tally_R, 2)
qt2_keyword_tally_R    = sort_dict_by_values(qt2_keyword_tally_R, 2)
qt2_abstract_tally_R   = sort_dict_by_values(qt2_abstract_tally_R, 2)
qt2_corpus_tally_R     = sort_dict_by_values(qt2_corpus_tally_R, 2)

qt2_title_tally_NR     = sort_dict_by_values(qt2_title_tally_NR, 2)
qt2_keyword_tally_NR   = sort_dict_by_values(qt2_keyword_tally_NR, 2)
qt2_abstract_tally_NR  = sort_dict_by_values(qt2_abstract_tally_NR, 2)
qt2_corpus_tally_NR    = sort_dict_by_values(qt2_corpus_tally_NR, 2)

In [15]:
def num_paper_per_keyword(terms, df):
    qt_count    = {}  # tally of how many papers each keyword appear in 

    qt_count_R  = {}  # tally of how many relevant papers each keyword appear in

    qt_count_NR = {}  # tally of how many not relevant papers each keyword appear in

    for term in terms:
        qt_count[term]         = 0
        qt_count_R[term]       = 0
        qt_count_NR[term]      = 0

    for term in terms:

        for index, row in df.iterrows():

            if term in row.all_text_corpus:
                if row.relevant==True:               
                    qt_count_R[term] +=1
                else:
                    qt_count_NR[term] +=1

                qt_count[term] +=1
    
    return sort_dict_by_values(qt_count, 1).copy(), sort_dict_by_values(qt_count_R, 1).copy(), sort_dict_by_values(qt_count_NR, 1).copy()    
    

In [16]:
qt1_count, qt1_count_R, qt1_count_NR = num_paper_per_keyword(query_terms1, text_df) 

In [17]:
qt2_count, qt2_count_R, qt2_count_NR = num_paper_per_keyword(query_terms2, text_df) 

In [18]:
with open('data/query_term_count.pkl', 'wb') as handle:
    pickle.dump(qt2_count, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/query_term_count_relevant.pkl', 'wb') as handle:
    pickle.dump(qt2_count_R, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/query_term_count_irrelevant.pkl', 'wb') as handle:
    pickle.dump(qt2_count_NR, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
normalize(get_num_relevant(text_df)[0], qt1_count_R)

{'atp': 0.989,
 'adenosine triphosphate': 0.874,
 'mechan stimul': 0.152,
 'mechan stress': 0.076,
 'mechan stretch': 0.065,
 'hypoton stress': 0.065,
 'mechan load': 0.054,
 'mechan deform': 0.051,
 'mechan strain': 0.04,
 'hydrostat pressur': 0.036,
 'mechan forc': 0.036,
 'hypoton shock': 0.029,
 'hypoton exposur': 0.022,
 'mechan perturb': 0.018,
 'membran stretch': 0.014,
 'hypoton stimul': 0.014,
 'hypoton swell': 0.011,
 'mechan pressur': 0.007,
 'mechan transduc': 0.004,
 'membran stress': 0.004,
 'hypoton stretch': 0.004,
 'osmotic stress': 0.004}

In [20]:
# qt2_count_R

In [21]:
normalize(get_num_relevant(text_df)[0], qt2_count_R)

{'atp': 0.989,
 'adenosine triphosphate': 0.874,
 'mechan': 0.726,
 'stimul': 0.534,
 'stress': 0.339,
 'membran': 0.249,
 'hypoton': 0.245,
 'stretch': 0.152,
 'mechan stimul': 0.152,
 'pressur': 0.148,
 'exposur': 0.137,
 'load': 0.116,
 'swell': 0.105,
 'mechan stress': 0.076,
 'forc': 0.072,
 'deform': 0.069,
 'strain': 0.065,
 'mechan stretch': 0.065,
 'hypoton stress': 0.065,
 'shock': 0.054,
 'mechan load': 0.054,
 'mechan deform': 0.051,
 'mechan strain': 0.04,
 'hydrostat': 0.036,
 'hydrostat pressur': 0.036,
 'mechan forc': 0.036,
 'compress': 0.036,
 'perturb': 0.029,
 'hypoton shock': 0.029,
 'hypoton exposur': 0.022,
 'transduc': 0.018,
 'mechan perturb': 0.018,
 'membran stretch': 0.014,
 'hypoton stimul': 0.014,
 'hypoton swell': 0.011,
 'displac': 0.011,
 'centrifug': 0.011,
 'mechan pressur': 0.007,
 'mechan transduc': 0.004,
 'membran stress': 0.004,
 'shrink': 0.004,
 'vibrat': 0.004,
 'hypoton stretch': 0.004,
 'osmotic stress': 0.004}

In [22]:
num_keywords1 = 0 # percentage of keywords that appear in any of the paper text
num_keywords_R1 = 0 # percentage of keywords that appear in relevent of the paper text
num_keywords_NR1 = 0 # percentage of keywords that appear in not relevent of the paper text

for key, value in qt1_count.items():
    if value > 0:
        num_keywords += 1
        
for key, value in qt1_count_R.items():
    if value > 0:
        num_keywords_R += 1
        
for key, value in qt1_count_NR.items():
    if value > 0:
        num_keywords_NR += 1
        
percent_kw = num_keywords / len(query_terms1)
percent_kw_R = num_keywords_R / len(query_terms1)
percent_kw_NR = num_keywords_NR / len(query_terms1)

print('percentage of keywords that appear in all papers:          {}'.format(percent_kw))
print('percentage of keywords that appear in relevant papers:     {}'.format(percent_kw_R))
print('percentage of keywords that appear in not relevant papers: {}'.format(percent_kw_NR))


NameError: name 'num_keywords' is not defined

In [23]:
num_keywords2 = 0 # percentage of keywords that appear in any of the paper text
num_keywords_R2 = 0 # percentage of keywords that appear in relevent of the paper text
num_keywords_NR2 = 0 # percentage of keywords that appear in not relevent of the paper text

for key, value in qt2_count.items():
    if value > 0:
        num_keywords += 1
        
for key, value in qt2_count_R.items():
    if value > 0:
        num_keywords_R += 1
        
for key, value in qt2_count_NR.items():
    if value > 0:
        num_keywords_NR += 1
        
percent_kw2 = num_keywords / len(query_terms2)
percent_kw_R2 = num_keywords_R / len(query_terms2)
percent_kw_NR2 = num_keywords_NR / len(query_terms2)

print('percentage of keywords2 that appear in the papers:          {}'.format(percent_kw2))
print('percentage of keywords2 that appear in relevant papers:     {}'.format(percent_kw_R2))
print('percentage of keywords2 that appear in not relevant papers: {}'.format(percent_kw_NR2))

NameError: name 'num_keywords' is not defined

In [24]:
# explore above cell more

In [25]:
# qt_count         = sort_dict_by_values(qt_count, 0)
# # qt_freq_count    = sort_dict_by_values(qt_freq_count, 0)

# qt_count_R       = sort_dict_by_values(qt_count_R, 0)
# # qt_freq_count_R  = sort_dict_by_values(qt_freq_count_R, 0)

# qt_count_NR      = sort_dict_by_values(qt_count_NR, 0)
# # qt_freq_count_NR = sort_dict_by_values(qt_freq_count_NR, 0)

here we'll determine which words appear in which class at least once

In [26]:
both1       = []
none1       = []
R_not_NR1   = []
NR_not_R1   = []
ctr = 1
for term in query_terms1:
    
    
    in_relevant = False
    in_not_relevant = False
    # if term appears in just once paper we mark it true
    for index, row in text_df.iterrows():        
        
        if term in row.all_text_corpus:
            
            if row.relevant == True:
                in_relevant = True
                
            else:
                in_not_relevant = True
                
    
    if in_relevant == True and in_not_relevant == True: # word appears in both classes at least once
        both1.append(term)
        
    elif in_relevant == False and in_not_relevant == False: # word never appears in either class
        none1.append(term)
    
    elif in_relevant == False and in_not_relevant == True: # word appears in non relevant papers at least once but never in relevant papers
        NR_not_R1.append(term)
        
    else:                    # word appears in relevant papers at least once but never in non relevant papers
        R_not_NR1.append(term)   
    
    print("processed {}/{} terms".format(ctr, len(query_terms1)))
    ctr+=1


processed 1/200 terms
processed 2/200 terms
processed 3/200 terms
processed 4/200 terms
processed 5/200 terms
processed 6/200 terms
processed 7/200 terms
processed 8/200 terms
processed 9/200 terms
processed 10/200 terms
processed 11/200 terms
processed 12/200 terms
processed 13/200 terms
processed 14/200 terms
processed 15/200 terms
processed 16/200 terms
processed 17/200 terms
processed 18/200 terms
processed 19/200 terms
processed 20/200 terms
processed 21/200 terms
processed 22/200 terms
processed 23/200 terms
processed 24/200 terms
processed 25/200 terms
processed 26/200 terms
processed 27/200 terms
processed 28/200 terms
processed 29/200 terms
processed 30/200 terms
processed 31/200 terms
processed 32/200 terms
processed 33/200 terms
processed 34/200 terms
processed 35/200 terms
processed 36/200 terms
processed 37/200 terms
processed 38/200 terms
processed 39/200 terms
processed 40/200 terms
processed 41/200 terms
processed 42/200 terms
processed 43/200 terms
processed 44/200 ter

In [27]:
both2       = []
none2       = []
R_not_NR2   = []
NR_not_R2   = []
ctr = 1
for term in query_terms2:
    
    
    in_relevant = False
    in_not_relevant = False
    # if term appears in just once paper we mark it true
    for index, row in text_df.iterrows():        
        
        if term in row.all_text_corpus:
            
            if row.relevant == True:
                in_relevant = True
                
            else:
                in_not_relevant = True
                
    
    if in_relevant == True and in_not_relevant == True: # word appears in both classes at least once
        both2.append(term)
        
    elif in_relevant == False and in_not_relevant == False: # word never appears in either class
        none2.append(term)
    
    elif in_relevant == False and in_not_relevant == True: # word appears in non relevant papers at least once but never in relevant papers
        NR_not_R2.append(term)
        
    else:                    # word appears in relevant papers at least once but never in non relevant papers
        R_not_NR2.append(term)   
    
    print("processed {}/{} terms".format(ctr, len(query_terms2)))
    ctr+=1


processed 1/230 terms
processed 2/230 terms
processed 3/230 terms
processed 4/230 terms
processed 5/230 terms
processed 6/230 terms
processed 7/230 terms
processed 8/230 terms
processed 9/230 terms
processed 10/230 terms
processed 11/230 terms
processed 12/230 terms
processed 13/230 terms
processed 14/230 terms
processed 15/230 terms
processed 16/230 terms
processed 17/230 terms
processed 18/230 terms
processed 19/230 terms
processed 20/230 terms
processed 21/230 terms
processed 22/230 terms
processed 23/230 terms
processed 24/230 terms
processed 25/230 terms
processed 26/230 terms
processed 27/230 terms
processed 28/230 terms
processed 29/230 terms
processed 30/230 terms
processed 31/230 terms
processed 32/230 terms
processed 33/230 terms
processed 34/230 terms
processed 35/230 terms
processed 36/230 terms
processed 37/230 terms
processed 38/230 terms
processed 39/230 terms
processed 40/230 terms
processed 41/230 terms
processed 42/230 terms
processed 43/230 terms
processed 44/230 ter

In [31]:
with open('data/words_in_neither.pkl', 'wb') as handle:
    pickle.dump(none2, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/words_in_both.pkl', 'wb') as handle:
    pickle.dump(both2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
  
print("words in both: {}".format(len(both1)))
for word in both1:
    print(word)
print('\n')

print("words in neither: {}\n".format(len(none1)))
for word in none1:
    print(word)
print('\n')

print("words in relevant papers but not irrelevant: {}\n".format(len(R_not_NR1)))
for word in R_not_NR1:
    print(word)
print('\n')

print("words in irrelevant paper but not relevant: {}\n".format(len(NR_not_R1)))
for word in NR_not_R1:
    print(word)
print('\n')

words in both: 22
mechan deform
hypoton swell
mechan transduc
mechan stimul
mechan load
adenosine triphosphate
membran stress
membran stretch
hydrostat pressur
hypoton stimul
mechan stretch
mechan pressur
mechan stress
mechan forc
hypoton exposur
hypoton stretch
mechan perturb
atp
hypoton stress
hypoton shock
mechan strain
osmotic stress


words in neither: 146

hydrostat shock
membran shrink
osmotic shrink
centrifug stretch
hyperto agit
centrifug exposur
hyperto shock
hydrostat load
fluid swell
membran vibrat
hydrostat stimul
gravit decompress
hyperto transduc
fluid decompress
mechan shrink
hyperto exposur
centrifug transduc
osmotic decompress
hydrostat decompress
centrifug load
gravit shrunk
hydrostat agit
fluid shrunk
hydrostat shrink
gravit press
hyperto stress
centrifug expose
osmotic strain
hyperto decompress
gravit perturb
gravit pressur
gravit shrink
hyperto stimul
mechan shrunk
hyperto perturb
fluid agit
hypoton vibrat
hyperto forc
centrifug deform
gravit swell
osmotic swell
h

In [33]:
  
print("words in both: {}".format(len(both2)))
for word in both2:
    print(word)
print('\n')

print("words in neither: {}\n".format(len(none2)))
for word in none2:
    print(word)
print('\n')

print("words in relevant papers but not irrelevant: {}\n".format(len(R_not_NR2)))
for word in R_not_NR2:
    print(word)
print('\n')

print("words in irrelevant paper but not relevant: {}\n".format(len(NR_not_R2)))
for word in NR_not_R2:
    print(word)
print('\n')

words in both: 44
mechan deform
hypoton swell
shock
mechan transduc
stretch
swell
mechan
hydrostat
mechan stimul
strain
mechan load
transduc
forc
adenosine triphosphate
perturb
membran stress
membran stretch
load
pressur
shrink
hydrostat pressur
hypoton stimul
exposur
mechan stretch
vibrat
displac
mechan pressur
deform
mechan stress
mechan forc
centrifug
stress
hypoton exposur
hypoton stretch
mechan perturb
hypoton
membran
atp
hypoton stress
compress
hypoton shock
stimul
mechan strain
osmotic stress


words in neither: 148

hydrostat shock
membran shrink
osmotic shrink
centrifug stretch
hyperto agit
centrifug exposur
hyperto shock
hydrostat load
fluid swell
membran vibrat
gravit decompress
hydrostat stimul
hyperto transduc
fluid decompress
mechan shrink
hyperto exposur
centrifug transduc
osmotic decompress
hydrostat decompress
centrifug load
gravit shrunk
hydrostat agit
fluid shrunk
hydrostat shrink
gravit press
hyperto stress
centrifug expose
osmotic strain
hyperto decompress
gravit p

plotting words that appear in most papers

check if papers had neither atp not adenosine triphosphate

In [34]:
no_atp       = 0
no_adenosine = 0
neither      = 0
neither_R    = 0
neither_NR   = 0

for index, row in text_df.iterrows():
    if 'atp' not in row.all_text_corpus:
        no_atp+=1
        
    if 'adenosine triphosphate' not in row.all_text_corpus:
        no_adenosine+=1

    if 'adenosine triphosphate' not in row.all_text_corpus and 'atp' not in row.all_text_corpus:
        neither +=1
        
        if row.relevant == True:
            neither_R +=1
        else:
            neither_NR +=1

In [35]:
print("number of irrelevant papers that don't mention ATP: {}".format(neither_NR))
print("number of relevant papers that don't mention ATP:   {}".format(neither_R))

number of irrelevant papers that don't mention ATP: 1307
number of relevant papers that don't mention ATP:   0


In [36]:
no_qt_tally1 = 0
no_qt_tally_R1 = 0
no_qt_tally_NR1 = 0

updated_df1 = []
for index, row in text_df.iterrows():
    has_qt = False
    
    for word in query_terms1:
        
        if word in row.all_text_corpus:
            has_qt=True

    if not has_qt: # if none of the query terms were present
        no_qt_tally1+=1
        if row.relevant==True:
            no_qt_tally_R1 +=1
        else:
            no_qt_tally_NR1 +=1
    else:
        updated_df1.append(row)
        
print("number of irrelevant papers that don't mention any query term: {}".format(no_qt_tally_NR1))
print("number of relevant papers that don't mention any query term: {}".format(no_qt_tally_R1))


number of irrelevant papers that don't mention any query term: 1045
number of relevant papers that don't mention any query term: 0


In [37]:

no_qt_tally2 = 0
no_qt_tally_R2 = 0
no_qt_tally_NR2 = 0

updated_df2 = []
for index, row in text_df.iterrows():
    has_qt = False
    
    for word in query_terms2:
        
        if word in row.all_text_corpus:
            has_qt=True

    if not has_qt: # if none of the query terms were present
        no_qt_tally2+=1
        if row.relevant==True:
            no_qt_tally_R2 +=1
        else:
            no_qt_tally_NR2 +=1
    else:
        updated_df2.append(row)
        
print("number of irrelevant papers that don't mention any query term: {}".format(no_qt_tally_NR2))
print("number of relevant papers that don't mention any query term: {}".format(no_qt_tally_R2))



number of irrelevant papers that don't mention any query term: 244
number of relevant papers that don't mention any query term: 0


In [38]:
text_df = pd.DataFrame(data = updated_df1)

In [39]:
text_df = text_df.sample(frac=1)


In [40]:
updated_df = []
ctr = 0
for index, row in text_df.iterrows():


    if 'adenosine triphosphate' in row.all_text_corpus or 'atp' in row.all_text_corpus:
        updated_df.append(row)
    else:
        ctr+=1

In [41]:
text_df = pd.DataFrame(data = updated_df)

In [42]:
text_df = text_df.reset_index(drop=True)

In [43]:
# qt_count, qt_count_R, qt_count_NR = num_paper_per_keyword(query_terms, text_df)

In [44]:
relevent, irrelevant = get_num_relevant(text_df)

In [45]:
print("relevant: {}".format(relevant))
print("irrelevant: {}".format(irrelevant))

relevant: True
irrelevant: 6878


In [46]:
qt_count_R_normalized = normalize(relevant, qt_count_R)
qt_count_NR_normalized = normalize(irrelevant, qt_count_NR)

NameError: name 'qt_count_R' is not defined

In [47]:
# qt_count_R_normalized

In [48]:
# qt_count_NR_normalized

In [49]:
def top_n_from_dict(dictionary, n):
    ctr = 0
    new_dict = {}
    for key in dictionary.keys():
        if ctr<n:
            new_dict[key] = dictionary[key]
        ctr +=1
    return new_dict
            

graphs and plots

In [50]:
atp = 0
adenosine = 0
both = 0
adenosine_no_atp = 0
atp_no_adenosine = 0
neither = 0

for row in text_df.loc[text_df['relevant'] == True].iterrows():
    row = row[1]
    if 'atp' in row.all_text_corpus and 'adenosine triphosphate' in row.all_text_corpus:

        both+=1
    elif 'atp' in row.all_text_corpus and 'adenosine triphosphate' not in row.all_text_corpus:
        atp_no_adenosine +=1
    
    elif 'atp' not in row.all_text_corpus and 'adenosine triphosphate' in row.all_text_corpus:
        adenosine_no_atp +=1
        
       
    else:
        neither+=1
        
    if 'atp' in row.all_text_corpus:
        atp+=1
    
    if 'adenosine triphosphate' in row.all_text_corpus:
        adenosine+=1
    

            
print("atp is in {} relevant papers".format(atp))  
print("adenosine triphosphate is in {} relevant papers".format(adenosine))
print("atp and adenosine triphosphate are in {} relevant papers".format(both))
print("atp but no adenosine triphosphate is in {} relevant papers".format(atp_no_adenosine))
print("adenosine triphosphate but no atp is in {} relevant papers".format(adenosine_no_atp))
print("neither atp nor adenosine triphosphate is in {} relevant papers".format(neither))

atp is in 274 relevant papers
adenosine triphosphate is in 242 relevant papers
atp and adenosine triphosphate are in 239 relevant papers
atp but no adenosine triphosphate is in 35 relevant papers
adenosine triphosphate but no atp is in 3 relevant papers
neither atp nor adenosine triphosphate is in 0 relevant papers


In [121]:
used_terms1 = [] # list of the original query terms that show up in the text
for word in query_terms1:
    if word not in none1:
        used_terms1.append(word)

In [51]:
used_terms2 = [] # list of the original query terms that show up in the text
for word in query_terms2:
    if word not in none2:
        used_terms2.append(word)


In [53]:
with open('data/used_query_terms.pkl', 'wb') as f:
    pickle.dump(used_terms2, f)

In [124]:
mechan_words1 = [word for word in used_terms1 if word != 'atp' and word != 'adenosine triphosphate']

In [125]:
mechan_words2 = [word for word in used_terms2 if word != 'atp' and word != 'adenosine triphosphate']

In [126]:
no_qt_tally1 = 0
no_qt_tally_R1 = 0
no_qt_tally_NR1 = 0
no_qt_R1 = []

updated_df = []
for index, row in text_df.iterrows():
    has_qt = False
    
    for word in mechan_words1:
        
        if word in row.all_text_corpus:
            has_qt=True

    if not has_qt: # if none of the query terms were present
        no_qt_tally1+=1
        if row.relevant==True:
            no_qt_tally_R1 +=1
            no_qt_R1.append(row) 
        else:
            no_qt_tally_NR1 +=1
    else:
        updated_df.append(row)

In [127]:
print("number of papers with none of the mechanical terms: {}".format(no_qt_tally1))
print("number of irrelevant papers with none of the mechanical terms: {}".format(no_qt_tally_NR1))
print("number of relevant papers with none of the mechanical terms: {}".format(no_qt_tally_R1))

number of papers with none of the mechanical terms: 5578
number of irrelevant papers with none of the mechanical terms: 5453
number of relevant papers with none of the mechanical terms: 125


In [128]:
no_qt_tally2 = 0
no_qt_tally_R2 = 0
no_qt_tally_NR2 = 0
no_qt_R2 = []

updated_df = []
for index, row in text_df.iterrows():
    has_qt = False
    
    for word in mechan_words2:
        
        if word in row.all_text_corpus:
            has_qt=True

    if not has_qt: # if none of the query terms were present
        no_qt_tally2+=1
        if row.relevant==True:
            no_qt_tally_R2 +=1
            no_qt_R2.append(row) 
        else:
            no_qt_tally_NR2 +=1
    else:
        updated_df.append(row)

In [129]:
print("number of papers with none of the mechanical terms: {}".format(no_qt_tally2))
print("number of irrelevant papers with none of the mechanical terms: {}".format(no_qt_tally_NR2))
print("number of relevant papers with none of the mechanical terms: {}".format(no_qt_tally_R2))

number of papers with none of the mechanical terms: 1074
number of irrelevant papers with none of the mechanical terms: 1065
number of relevant papers with none of the mechanical terms: 9


# for this study we've been able to remove 2000 irrelevant and only 9 relevant papers

In [130]:
# looking at the papers that don't mention any of the mechanical words
for row in no_qt_R2:
    for word in mechan_words2:
        
        if word in row.all_text_corpus:
            print("{} found".format(word))
    print(row['original abstract'])
    print('\n')
 
# note try unigram keywords, only keep papers whose keywords appear near sentences with atp (1 or 2 sentence range)

We have shown previously that wounding of human corneal epithelial (HCE) cells resulted in epidermal growth factor receptor (EGFR) transactivation through ectodomain shedding of heparin-binding EGF-like growth factor (HB-EGF). However, the initial signal to trigger these signaling events in response to cell injury remains elusive. In the present study, we investigated the role of ATP released from the injured cells in EGFR transactivation in HCE cells as well as in BEAS 2B cells, a bronchial epithelial cell line. Wounding of epithelial monolayer resulted in the release of ATP into the culture medium. The wound-induced rapid activation of phosphatidylinositol-3-kinase (PI3K) and extracellular signal-regulated kinase (ERK) pathways in HCE cells was attenuated by eliminating extracellular ATP, ADP and adenosine. The nonhydrolyzable ATP analog ATP-gamma-S induced rapid and sustained EGFR activation that depended on HB-EGF shedding and ADAM (a disintegrin and metalloproteinase). Targeting p

In [54]:
atp_count = []

for row in text_df.iterrows():
    row = row[1]
    atp = 0
    atp+=row.all_text.count('atp')
    adenosine = 0
    adenosine+=row.all_text.count('adenosine triphosphate')
    atp_count.append((atp, adenosine, row.relevant))



In [55]:
atp_count_R  = [tup for tup in atp_count if  tup[-1]==True]
atp_count_NR = [tup for tup in atp_count if tup[-1]==False]


In [56]:
atp_count_R

[(36, 0, True),
 (23, 1, True),
 (29, 1, True),
 (6, 0, True),
 (39, 1, True),
 (21, 1, True),
 (17, 0, True),
 (24, 1, True),
 (12, 1, True),
 (11, 1, True),
 (33, 1, True),
 (29, 1, True),
 (24, 1, True),
 (12, 1, True),
 (3, 1, True),
 (12, 1, True),
 (20, 1, True),
 (9, 1, True),
 (15, 0, True),
 (23, 1, True),
 (12, 1, True),
 (15, 0, True),
 (18, 1, True),
 (12, 1, True),
 (15, 1, True),
 (31, 1, True),
 (21, 1, True),
 (27, 1, True),
 (30, 1, True),
 (9, 0, True),
 (30, 1, True),
 (21, 1, True),
 (38, 1, True),
 (48, 1, True),
 (15, 1, True),
 (3, 0, True),
 (15, 1, True),
 (45, 1, True),
 (36, 1, True),
 (32, 1, True),
 (12, 0, True),
 (17, 1, True),
 (42, 1, True),
 (21, 1, True),
 (15, 1, True),
 (24, 1, True),
 (12, 1, True),
 (0, 1, True),
 (21, 1, True),
 (33, 1, True),
 (43, 1, True),
 (13, 1, True),
 (0, 1, True),
 (27, 1, True),
 (12, 0, True),
 (30, 1, True),
 (30, 1, True),
 (15, 1, True),
 (27, 1, True),
 (0, 1, True),
 (33, 1, True),
 (14, 1, True),
 (45, 1, True),


In [57]:
rows_by_atp_count = {}
for num in range(20):
    
    true = 0
    false = 0
    row_count = 0
    for row in atp_count:
        
        if row[0] == num:
            row_count+=1
            if row[-1]==True:
                true+=1
            else:
                false+=1
    rows_by_atp_count[num] = (true, false)
        

In [ ]:
# rows_by_atp_count